# Examen Data science 1
## 2023 - P4

Naam: Elias De Hondt
Klas: INF105A
Studentnummer: 0160712-80

In [60]:
# Zet hier al je imports:
import pandas as pd
import numpy as np
from IPython import InteractiveShell
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from scipy.stats import linregress
from sklearn.metrics import mean_squared_error
from W20P4.forecast import seasonal_decomposition_forecast
from statsmodels.tsa.seasonal import seasonal_decompose
from W20P4.forecast import find_period, plot_seasonal_decompositon, create_trend_model, predict, forecast_errors, GeneralRegression, linear_combination
import graphviz
from id3 import Id3Estimator, export_graphviz
from matplotlib import pyplot as plt
import matplotlib
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.spatial.distance import cdist
from sklearn.cluster import AgglomerativeClustering
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from W23P4.scripts.pca_hulpfuncties import plot_pca
from pca import pca

In [61]:
# Zet hier alle functies die je wil gebruiken:
def moving_average(y: np.array, m=4):
    if y.size < m:
        return np.nan

    return np.mean(y[-m:])

def entropy(series: pd.Series):
    vc = series.value_counts(normalize=True, sort=False)
    return -(vc * np.log2(vc)).sum()

def information_gain(parent_table: pd.DataFrame, attribute: str, target: str):
    # bepaal entropie van parent table
    entropy_parent = entropy(parent_table[target])
    child_entropies = []
    child_weights = []

    # bereken entropies of child tables
    for (label, fraction) in parent_table[attribute].value_counts().items():
        child_df = parent_table[parent_table[attribute] == label]
        child_entropies.append(entropy(child_df[target]))
        child_weights.append(int(fraction))

    # calculate the difference between parent entropy and weighted child entropies
    return entropy_parent - np.average(child_entropies, weights=child_weights)

Hier staat code om het bestand in te lezen.  Pas eventueel het path aan.  De rest van de code zou zo moeten werken:

In [62]:
# Voorgeschreven code:
examen = pd.read_csv('examen_2023_P4.csv')
examen.grootte = pd.Categorical(examen.grootte, categories=('klein', 'normaal', 'groot'), ordered=True)
examen.tevredenheid = pd.Categorical(examen.tevredenheid, categories=('teleurgesteld', 'licht ontevreden', 'ok', 'tevreden', 'heel tevreden'), ordered=True)
examen.belangrijkheid = pd.Categorical(examen.belangrijkheid, categories=('niet belangrijk', 'beetje belangrijk', 'belangrijk', 'heel belangrijk'), ordered=True)
examen.behaalde_graad = pd.Categorical(examen.behaalde_graad, categories=('gebuisd', 'voldoening', 'onderscheiding', 'grote onderscheiding', 'grootste onderscheiding'), ordered=True)
display(examen)
# De code doet dus het volgende:  Het zet de kolommen om naar een categorie en geeft de volgorde van de categorie mee.  Dit is nodig voor de ID3 en ook k-means algoritme. En zorgt ervoor dat de data niet als een nummer wordt gezien maar als een categorie. Dit is nodig voor de ID3 en ook k-means algoritme. (extra punt? :) ).
# Dus Kolommen "grootte", "tevredenheid", "belangrijkheid" en "behaalde_graad" worden omgezet naar een categorie.

,klantNummer,grootte,waardering,totaal_bedrag,bedrag,lengte,oplaadtijd,concentratie,beltijd,tevredenheid,belangrijkheid,behaalde_graad,saldo
0,32814,normaal,3,8109.12,1556.30,194.4,199.7,54.51,136.8,teleurgesteld,heel belangrijk,gebuisd,-71.26
1,32856,normaal,3,9481.44,1339.01,172.1,165.3,44.67,131.0,licht ontevreden,belangrijk,voldoening,95.40
2,32041,normaal,3,8515.58,1480.33,160.6,188.5,42.37,127.6,tevreden,beetje belangrijk,onderscheiding,-162.92
3,32739,klein,2,7881.27,1736.86,178.1,208.3,47.25,130.8,ok,heel belangrijk,onderscheiding,9.92
4,32757,normaal,2,7095.56,1802.29,160.9,214.8,49.08,135.3,ok,beetje belangrijk,gebuisd,214.24
5,32359,klein,2,9047.55,1606.87,178.0,177.6,52.62,143.2,tevreden,beetje belangrijk,grote onderscheiding,-108.81
6,32274,normaal,4,8395.51,1471.27,174.2,190.9,52.65,141.8,tevreden,belangrijk,voldoening,137.52
7,32012,normaal,2,7901.42,1619.80,184.8,198.1,43.05,137.6,ok,heel belangrijk,onderscheiding,6.00
8,32908,klein,1,8368.78,1599.27,173.3,196.6,52.36,133.9,tevreden,beetje belangrijk,voldoening,54.19
9,32365,normaal,2,9245.22,1601.39,159.1,169.1,41.52,136.3,ok,belangrijk,onderscheiding,216.55


## Vraag 1

antwoorden:
a) **0.703083**
b) **0.703083**
c) **Dus C**
d) **-0.9001109182755574**
e) **De correlatiecoëfficiënt is een statistische maat die de sterkte en richting van de lineaire relatie tussen twee variabelen aangeeft. Dus nee, de correlatie is te laag.**

In [63]:
# a) & b)
TEMP = examen[['klantNummer', 'waardering', 'totaal_bedrag', 'bedrag', 'lengte', 'oplaadtijd', 'concentratie', 'saldo']]
corr = TEMP.corr()
display(corr) # 0.703083

# c)
# Dus C

# d)
corr = np.corrcoef(examen.oplaadtijd, examen.totaal_bedrag)[0, 1]
display(corr)

# e)
# De correlatiecoëfficiënt is een statistische maat die de sterkte en richting van de lineaire relatie tussen twee variabelen aangeeft.
# Dus nee, de correlatie is te laag.

,klantNummer,waardering,totaal_bedrag,bedrag,lengte,oplaadtijd,concentratie,saldo
klantNummer,1.000000,0.078869,-0.212889,0.219002,0.109729,0.203819,0.158211,0.009933
waardering,0.078869,1.000000,-0.102473,-0.032464,0.307897,0.045171,0.076863,0.247219
totaal_bedrag,-0.212889,-0.102473,1.000000,-0.800002,-0.211897,-0.900111,0.167332,-0.011340
bedrag,0.219002,-0.032464,-0.800002,1.000000,0.200211,0.703083,-0.165064,0.037204
lengte,0.109729,0.307897,-0.211897,0.200211,1.000000,0.171829,0.499936,-0.060052
oplaadtijd,0.203819,0.045171,-0.900111,0.703083,0.171829,1.000000,-0.117171,-0.045437
concentratie,0.158211,0.076863,0.167332,-0.165064,0.499936,-0.117171,1.000000,-0.058147
saldo,0.009933,0.247219,-0.011340,0.037204,-0.060052,-0.045437,-0.058147,1.000000


-0.9001109182755574

## Vraag 2

antwoorden:
a) **[182.47954633 165.14082597 176.41749752 196.88738537 202.10839173
 186.51479456 175.69455212 187.54654776 185.90835029 186.07751632
 176.54357409 171.16537099 182.51784807 188.560746   160.4512563
 185.69689275 184.26376919 181.99359296 178.02696868 154.35808733
 179.29571393 195.19013938 183.13386777 140.33246776 185.08805462
 190.0577058  164.31255078 180.84454066 167.00245028 187.62953487
 180.5740346  192.5744495  183.5152893  161.1215368  173.74036526
 191.87943246 149.4147686  207.22087649 194.9403801  183.49693638
 170.72809276 193.2822338  189.6044685  187.24970925 190.20213529
 184.55980975 176.03448009 183.89351901 190.53887145 174.19440051
 179.31167299 186.87706521 182.87453305 177.94717339 174.58220566
 197.81301083 187.18587301 190.90513186 194.10651923 172.29048469]**
b)
c) **0.4943250544126929**
d) **Als de R²-waarde van de regressie dicht bij 1 ligt, geeft dit aan dat er een sterke lineaire relatie is tussen de variabelen. Variantie is een statistische maat die de spreiding van een dataset rondom het gemiddelde beschrijft, terwijl R² de proportie van de variantie van de afhankelijke variabele verklaard door de regressievergelijking weergeeft. Dus [A]**
e)
f) **Dus B**

In [64]:
# a)
model = LinearRegression()

X = examen[['bedrag']]
y = examen.oplaadtijd
model.fit(X, y)
y_hat = model.predict(X) # Voorspellingen maken.
print(y_hat) # y_hat = Voorspellingen.
# b)


# c)
print(r2_score(y, y_hat))

# d)
# Als de R²-waarde van de regressie dicht bij 1 ligt, geeft dit aan dat er een sterke lineaire relatie is tussen de variabelen. Variantie is een statistische maat die de spreiding van een dataset rondom het gemiddelde beschrijft, terwijl R² de proportie van de variantie van de afhankelijke variabele verklaard door de regressievergelijking weergeeft. Dus [A]

# e)


# f)
# Dus B

LinearRegression()

[182.47954633 165.14082597 176.41749752 196.88738537 202.10839173
 186.51479456 175.69455212 187.54654776 185.90835029 186.07751632
 176.54357409 171.16537099 182.51784807 188.560746   160.4512563
 185.69689275 184.26376919 181.99359296 178.02696868 154.35808733
 179.29571393 195.19013938 183.13386777 140.33246776 185.08805462
 190.0577058  164.31255078 180.84454066 167.00245028 187.62953487
 180.5740346  192.5744495  183.5152893  161.1215368  173.74036526
 191.87943246 149.4147686  207.22087649 194.9403801  183.49693638
 170.72809276 193.2822338  189.6044685  187.24970925 190.20213529
 184.55980975 176.03448009 183.89351901 190.53887145 174.19440051
 179.31167299 186.87706521 182.87453305 177.94717339 174.58220566
 197.81301083 187.18587301 190.90513186 194.10651923 172.29048469]
0.4943250544126929


## Vraag 3

antwoorden:
a) **180.88**
b) **183.920809**
c) **120.88**
d) **Dus A**
e) **1**
f) **132.57386**
g) **Index 4**

In [65]:
# a)
moving_average = moving_average(examen.beltijd, m=5)
print(moving_average) # 180.88

# b)
PredictionData = pd.DataFrame(predict(examen.beltijd, 54, 62, linear_combination)[61:], columns=['linear_combination'], index=range(61, 62)).transpose()
# predict met data 54 tot 62 (laatste 5 waarden)
display(PredictionData) # 183.920809

# c)
errors = forecast_errors(examen.beltijd.size, moving_average, method='moving_average')
display(errors) # 120.88

# d)
# Dus [A]

# e)
m = find_period(examen.beltijd)
print('m =', m)
# Periode is 1, maar dit hier geen echte periode, want er is geen herhaling.
# f)
sd_model = seasonal_decompose(examen.saldo, model='additive', period=10)
display(sd_model.seasonal) # 132.57386

# g)
# plot_seasonal_decompositon(sd_model, 'additive', figsize=(15, 15))
print(sd_model.seasonal.max())

180.88


,61
linear_combination,183.920809


,MAE,RMSE,MAPE
moving_average,120.88,120.88,2.014667


m = [1]


0     -21.15944
1     132.57386
2    -148.79944
3       2.74446
4     179.07926
5    -150.11034
6      61.33536
7     -89.42654
8     -67.93464
9     101.69746
10    -21.15944
11    132.57386
12   -148.79944
13      2.74446
14    179.07926
15   -150.11034
16     61.33536
17    -89.42654
18    -67.93464
19    101.69746
20    -21.15944
21    132.57386
22   -148.79944
23      2.74446
24    179.07926
25   -150.11034
26     61.33536
27    -89.42654
28    -67.93464
29    101.69746
30    -21.15944
31    132.57386
32   -148.79944
33      2.74446
34    179.07926
35   -150.11034
36     61.33536
37    -89.42654
38    -67.93464
39    101.69746
40    -21.15944
41    132.57386
42   -148.79944
43      2.74446
44    179.07926
45   -150.11034
46     61.33536
47    -89.42654
48    -67.93464
49    101.69746
50    -21.15944
51    132.57386
52   -148.79944
53      2.74446
54    179.07926
55   -150.11034
56     61.33536
57    -89.42654
58    -67.93464
59    101.69746
Name: seasonal, dtype: float64

179.07925999999998


## Vraag 4

antwoorden:
a)
**Entropy grootte = 1.3059524812806642**

**Entropy tevredenheid = 1.726739
Entropy belangrijkheid = 1.663814
Entropy behaalde_graad = 2.041086**
b) **lengte**
c) **1.3059524812806642**
d)
e) zie code, stel de max_depth gelijk aan 3
f)

In [66]:
# Voorgeschreven code:
examen2 = examen[['tevredenheid', 'belangrijkheid', 'behaalde_graad']]
display(examen2.head())

,tevredenheid,belangrijkheid,behaalde_graad
0,teleurgesteld,heel belangrijk,gebuisd
1,licht ontevreden,belangrijk,voldoening
2,tevreden,beetje belangrijk,onderscheiding
3,ok,heel belangrijk,onderscheiding
4,ok,beetje belangrijk,gebuisd


In [67]:
# Voorgeschreven code:
examen3 = examen[['bedrag', 'lengte', 'saldo']]
display(examen3.head())

,bedrag,lengte,saldo
0,1556.30,194.4,-71.26
1,1339.01,172.1,95.40
2,1480.33,160.6,-162.92
3,1736.86,178.1,9.92
4,1802.29,160.9,214.24


In [68]:
# a)
E_ouder = entropy(examen.grootte)
print('Entropy, E(S) =', E_ouder)
print("\n")
E_ouder = examen2.apply(entropy)
print(E_ouder)
# e(tabel) = soma(x) -(p/n) * log^2(p/n)
# -(vc * np.log(vc) / np.log(base)).sum()

# b) | c)
IG_lengte = information_gain(examen, 'lengte', 'grootte')
IG_oplaadtijde = information_gain(examen, 'oplaadtijd', 'grootte')
IG_concentratie = information_gain(examen, 'concentratie', 'grootte')
IG_beltijd = information_gain(examen, 'beltijd', 'grootte')
IG_tevredenheid = information_gain(examen, 'tevredenheid', 'grootte')
IG_belangrijkheid = information_gain(examen, 'belangrijkheid', 'grootte')
IG_behaalde_graad = information_gain(examen, 'behaalde_graad', 'grootte')
IG_saldo = information_gain(examen, 'saldo', 'grootte')

print(IG_lengte)
print(IG_oplaadtijde)
print(IG_concentratie)
print(IG_beltijd)
print(IG_tevredenheid)
print(IG_belangrijkheid)
print(IG_behaalde_graad)
print(IG_saldo)

# d)


# e)


# f)



Entropy, E(S) = 1.3059524812806642


tevredenheid      1.726739
belangrijkheid    1.663814
behaalde_graad    2.041086
dtype: float64


C:\Users\elias\Downloads\DataScience1\venv\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\elias\Downloads\DataScience1\venv\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\elias\Downloads\DataScience1\venv\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\elias\Downloads\DataScience1\venv\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\elias\Downloads\DataScience1\venv\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\elias\Downloads\

1.3059524812806642
1.2726191479473308
1.3059524812806642
1.2726191479473308
0.12740725854983115
0.045291845666193264
0.07842940493874684
1.3059524812806642


C:\Users\elias\Downloads\DataScience1\venv\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\elias\Downloads\DataScience1\venv\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\elias\Downloads\DataScience1\venv\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\elias\Downloads\DataScience1\venv\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\elias\Downloads\DataScience1\venv\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\elias\Downloads\

## Vraag 5

antwoorden:
a) **175.30**
b) **20**
c) **20**


In [69]:
# Voorgeschreven code:
examen4 = examen[['bedrag', 'lengte', 'saldo']]
display(examen4)

,bedrag,lengte,saldo
0,1556.30,194.4,-71.26
1,1339.01,172.1,95.40
2,1480.33,160.6,-162.92
3,1736.86,178.1,9.92
4,1802.29,160.9,214.24
5,1606.87,178.0,-108.81
6,1471.27,174.2,137.52
7,1619.80,184.8,6.00
8,1599.27,173.3,54.19
9,1601.39,159.1,216.55


In [70]:
# a)
afstand_manhattan = pd.DataFrame(cdist(examen4, examen4, metric='cityblock'), columns=examen4.index.values, index=examen4.index.values)
display(afstand_manhattan) # 175.30

# b)
# afstanden_euclidische = pd.DataFrame(cdist(examen4, examen4, metric='euclidean'), columns=examen4.index, index=examen4.index)
#
# colors=list(matplotlib.colors.cnames.keys())[0:100:2] # Kleuren voor de dendrogram
#
# fig,ax=plt.subplots(figsize=(12,5))
# distances_single = linkage(afstanden_euclidische, method='single') # Single linkage (kleinste afstand eerst)
# _=ax.set_title('Dendrogram met Single Linkage')
# _=ax.set_xlabel('punt')
# _=ax.set_ylabel('Euclidischeafstand')
# _=ax.grid(linestyle='--' ,axis='y')
#
# dgram = dendrogram(distances_single, labels=afstanden_euclidische.index.values, link_color_func=lambda x: colors[x], ax=ax)

cluster = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='single')
cluster.fit(examen4)
examen4['cluster_with_3'] = cluster.labels_
display(examen4)

# c)
model = KMeans(n_clusters=2, n_init='auto', max_iter=100)
model.fit(examen4)
model_simpsons_Kmeans = pd.DataFrame(zip(examen4.saldo, examen4.bedrag, model.labels_),columns=["x", "y", "cluster"])
display(model_simpsons_Kmeans)


,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,0.00,406.25,201.43,278.04,564.99,104.52,314.01,150.36,189.52,368.20,...,1014.08,1200.92,894.43,1111.15,1412.99,1145.18,1219.16,1150.38,1223.64,1390.74
1,406.25,0.00,411.14,489.33,593.32,477.97,176.48,382.89,302.67,396.53,...,978.41,1273.85,932.16,1055.08,1243.38,1199.31,1247.49,1178.71,1282.77,1182.49
2,201.43,411.14,0.00,446.87,699.42,198.05,323.10,332.59,348.75,502.03,...,1106.91,1402.35,1060.66,1183.58,1394.88,1327.81,1355.99,1293.61,1411.27,1414.43
3,278.04,489.33,446.87,0.00,286.95,248.82,397.09,127.68,186.66,361.10,...,1100.56,1206.38,967.61,1211.43,1496.07,880.94,1184.28,1022.28,1034.10,1480.62
4,564.99,593.32,699.42,286.95,0.00,535.57,421.04,414.63,375.47,205.01,...,978.87,1084.69,843.32,1089.74,1339.98,736.05,1030.59,874.99,912.41,1358.93
5,104.52,477.97,198.05,248.82,535.57,0.00,385.73,134.54,175.30,349.74,...,1089.40,1204.30,956.25,1200.27,1484.71,1129.76,1189.74,1120.96,1213.22,1469.46
6,314.01,176.48,323.10,397.09,421.04,385.73,0.00,290.65,212.23,224.25,...,801.93,1097.37,755.68,878.60,1098.98,1022.83,1075.21,1006.43,1106.29,1091.33
7,150.36,382.89,332.59,127.68,414.63,134.54,290.65,0.00,80.22,254.66,...,990.72,1086.54,861.17,1091.59,1389.63,995.22,1077.84,1000.02,1078.68,1367.38
8,189.52,302.67,348.75,186.66,375.47,175.30,212.23,80.22,0.00,178.68,...,923.50,1053.60,787.95,1034.37,1309.41,979.06,1029.64,960.86,1062.52,1303.56
9,368.20,396.53,502.03,361.10,205.01,349.74,224.25,254.66,178.68,0.00,...,777.46,903.32,641.91,888.33,1134.97,828.78,856.96,794.58,912.24,1157.52


C:\Users\elias\Downloads\DataScience1\venv\lib\site-packages\sklearn\cluster\_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


AgglomerativeClustering(affinity='euclidean', linkage='single', n_clusters=3)

C:\Users\elias\AppData\Local\Temp\ipykernel_9988\328195160.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  examen4['cluster_with_3'] = cluster.labels_


,bedrag,lengte,saldo,cluster_with_3
0,1556.30,194.4,-71.26,0
1,1339.01,172.1,95.40,0
2,1480.33,160.6,-162.92,0
3,1736.86,178.1,9.92,0
4,1802.29,160.9,214.24,0
5,1606.87,178.0,-108.81,0
6,1471.27,174.2,137.52,0
7,1619.80,184.8,6.00,0
8,1599.27,173.3,54.19,0
9,1601.39,159.1,216.55,0


KMeans(max_iter=100, n_clusters=2, n_init='auto')

,x,y,cluster
0,-71.26,1556.30,0
1,95.40,1339.01,0
2,-162.92,1480.33,0
3,9.92,1736.86,0
4,214.24,1802.29,0
5,-108.81,1606.87,0
6,137.52,1471.27,0
7,6.00,1619.80,0
8,54.19,1599.27,0
9,216.55,1601.39,0


## Vraag 6

antwoorden:
a) **waardering, bedrag, lengte, oplaadtijd, concentratie, beltijd, saldo** **Alleen kolommen met nutege data**
b) **zie code**
c) **0.77324315%**
d) **De verklaarde variantie per hoofdcomponent.**
e) **PC1 = 0.259568 PC2 = -0.126818**

In [71]:
# a)
# waardering, bedrag, lengte, oplaadtijd, concentratie, beltijd, saldo Alleen kolommen met nutege data

# b)
examenX = examen.drop(columns=['grootte', 'tevredenheid', 'belangrijkheid', 'behaalde_graad', 'totaal_bedrag', 'klantNummer'])
display(examenX)

# c)
model = pca(normalize=True)
out = model.fit_transform(examenX, verbose=False)
display(out['explained_var'])

# d)
# De verklaarde variantie per hoofdcomponent.

# e)
display(out['topfeat'])
# PC1 = 0.259568 PC2 = -0.126818

,waardering,bedrag,lengte,oplaadtijd,concentratie,beltijd,saldo
0,3,1556.30,194.4,199.7,54.51,136.8,-71.26
1,3,1339.01,172.1,165.3,44.67,131.0,95.40
2,3,1480.33,160.6,188.5,42.37,127.6,-162.92
3,2,1736.86,178.1,208.3,47.25,130.8,9.92
4,2,1802.29,160.9,214.8,49.08,135.3,214.24
5,2,1606.87,178.0,177.6,52.62,143.2,-108.81
6,4,1471.27,174.2,190.9,52.65,141.8,137.52
7,2,1619.80,184.8,198.1,43.05,137.6,6.00
8,1,1599.27,173.3,196.6,52.36,133.9,54.19
9,2,1601.39,159.1,169.1,41.52,136.3,216.55


array([0.2862623 , 0.54414061, 0.77324315, 0.89458805, 0.95119387,
       0.98783968, 1.        ])

,PC,feature,loading,type
0,PC1,saldo,0.682748,best
1,PC2,oplaadtijd,-0.652011,best
2,PC3,concentratie,-0.641497,best
3,PC4,waardering,0.822911,best
4,PC5,lengte,0.635637,best
5,PC2,bedrag,-0.644692,weak
6,PC1,beltijd,0.672622,weak
